In [42]:
import pandas as pd
import numpy as np
import re 


In [43]:
# regular expression for validating an Email 
#Logic Copy Pasta from here for the email regex expression. It was the best working one that worked across 90k plus emails
#https://emailregex.com/
regex = r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)"

  


# We will be checking the row length after each operation for better clarity regarding the number of ops performed

## After importing the data file the Order of Operations are


1. Dropping all Empty Fields
2. Dropping all Fields with Multiple Empty Characters
3. Changing all to lowercase
4. Splitting and extracting multiple emails from the fields
5. Ensuring emails are unique in each field
6. Creating new rows for multi emails
7. Dropping all duplicates
8. Email Validation. If valid- > leave as it is. If not. Drop the row






In [44]:
raw_data = pd.read_excel("lever.xlsx")
df = pd.DataFrame(raw_data)
row_length, _ = df.shape
print("Row length -->",row_length)

Row length --> 96965


### Dropping all the Empty Fields below and also the ones with multiple empty characters

In [45]:
#Checking for empty and throwing away value
#Here NaN represents empty or you can say null
df.dropna(subset=['Email'],inplace=True)
#ALso check for empty strings after this. Just in case
#Below is code for regex check for longer empty string like '     '
df.replace(r'^\s*$', np.nan, regex=True)
df.dropna(subset=["Email"],inplace=True)
row_length, _ = df.shape
print("Row length -->",row_length)

Row length --> 95728


### Change all to lowercase

In [46]:
df['Email'] = df['Email'].str.lower()

## Few helper functions that come in handy later on for data transformations

In [47]:
#Creating a few helper functions for repeated use
#Trims and splits based on spaces
def despacer_decomma(x):
    x = x.strip()
    x = x.split(",")
    x = " ".join(x)
    x = x.split(" ")
    return x
'''
For removing false positives and weird chars in btwn valid emails, if the user uses some weird kind of characters for separating his email like ;; or -x- or ---- 
The smallest email can be of the size 6 characters. Using that heuristic we use the below logic to clear all the false positives
'''
def remover(x):
    if len(x) < 6:
        return None
    else:
        return x

def size_based_email_cleaner(l):
    l = list(map(remover,l))
    l = list(filter(None,l))
    return l

# Define a function for 
# for validating an Email 
def check(email):  
    # pass the regular expression 
    # and the string in search() method 
    if(re.search(regex,email)):  
        return True
          
    else:  
        return False 

### Email Splitting and Cleaning with the same field. Also remove case insensitive duplicates here

In [48]:
for i in df.index:
    val = df.at[i, "Email"]
    val1 = despacer_decomma(val)
    val2 = list(set(size_based_email_cleaner(val1)))
    df.at[i, "Email"] = val2

### Taking those multiple emails and creating multiple entries from them

In [50]:
for i in df.index:
    val = df.at[i, "Email"]
    if len(val)>1:
        core = val[0]
        if type(val) is type(" "):
            continue
        while len(val) > 1:
            temp_val = val.pop()
            new_df = df.loc[i, :]
            new_df.at["Email"] = str(temp_val)
            df = df.append(new_df,ignore_index=True)
    #Uncomment below row to see each email that is appended and new row that is created
   #         print("New Row Duplicated with Following Email :-",new_df["Email"])
        df.at[i,"Email"] = core
#Converting the list datatype in the col to a string one
for i in df.index:
    df.at[i, "Email"] = "".join(df.at[i, "Email"])
row_length, _ = df.shape
print("Row length -->",row_length)

Row length --> 103062


### Dropping all the duplicate values from the entire sheets with Email as Primary Key

In [51]:
df = df.drop_duplicates(subset = ["Email"])
row_length, _ = df.shape
print("Row length -->",row_length)

Row length --> 94939


### Dropping all the cells where the email is invalid

In [52]:
for i in df.index:
    val = df.at[i, "Email"]
    if check(val):
        continue
    else:
        print("This email is invalid",val)
        df.at[i, "Email"] = np.nan
print("Cleaning Completed")
df.dropna(subset=['Email'],inplace=True)
row_length, _ = df.shape
print("Row length -->",row_length)

This email is invalid 413-301-6544/jjzipagan@gmail.com
This email is invalid maggiedrc@comcast
This email is invalid olgawiley@kk
This email is invalid krvssiewoods@gmailcom
This email is invalid 978-609-6964|htruong@vcom.vt.edu
This email is invalid pillai22@redifcom
This email is invalid mesyoung@gmail
This email is invalid aabreu@clinicasmidoctor
This email is invalid caphr@earthlink
This email is invalid ?faizakemc@gmail.com
This email is invalid ?gruscinska@yahoo.com
This email is invalid ?docdeepa_rani128@yahoo.com
This email is invalid ?ilovedoctors@msn.com
This email is invalid ?bkoperski@llu.edu
This email is invalid ?vipmody@yahoo.com
This email is invalid ?soymerca@yahoo.com
This email is invalid ?jshostetter@gmail.com
This email is invalid ?mohsinhaseeb@hotmail.com
This email is invalid ?jwatson@mmc.edu
This email is invalid ?naureenzafarmd@gmail.com
This email is invalid ?monica.agarwal@umassmemorial.org
This email is invalid ?milindawalemd@gmail.com
This email is invalid 

### Saving the file to the memory

In [53]:
df.to_excel("vohra_phase1.xlsx",index=False)

#### Disclaimer :- For running this out of the box :- delete all column headers before calling the file, and paste back after wrangling. There is some weird bug that corrupts the dataframe in pandas